In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from xgboost import XGBClassifier


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [4]:
df = pd.read_csv('train.csv')

C:\Users\FAMILY\AppData\Local\Temp\ipykernel_31676\2436019669.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [6]:
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [7]:
df['Age'].unique() # i saw the whole unique values and there are alot of errors so we will drop this column

array(['23', '-500', '28_', ..., '4808_', '2263', '1342'], dtype=object)

In [8]:
df['Annual_Income'].unique() #it appears some values has _ at the end of it so we will remove that then change it to float

array(['19114.12', '34847.84', '34847.84_', ..., '20002.88', '39628.99',
       '39628.99_'], dtype=object)

In [9]:
df['Annual_Income'] = df['Annual_Income'].str.replace("_", '')
df['Annual_Income'] = df['Annual_Income'].astype(float)

In [10]:
mean_salary_by_customer = df.groupby('Customer_ID')['Monthly_Inhand_Salary'].transform('mean')
df['Monthly_Inhand_Salary'] = df['Monthly_Inhand_Salary'].fillna(mean_salary_by_customer)
# here we filled missing values with the mean of each customer

In [11]:
df['Type_of_Loan'] = df.groupby('Credit_Score')['Type_of_Loan'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None) 
# i filled the type of loan with the mode of each credit score

In [12]:
df['Month'].unique() # no problem here

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August'], dtype=object)

In [13]:
df['Num_of_Loan'].unique() # we see there are alot of unreasonable numbers in this column so we will drop the column

array(['4', '1', '3', '967', '-100', '0', '0_', '2', '3_', '2_', '7', '5',
       '5_', '6', '8', '8_', '9', '9_', '4_', '7_', '1_', '1464', '6_',
       '622', '352', '472', '1017', '945', '146', '563', '341', '444',
       '720', '1485', '49', '737', '1106', '466', '728', '313', '843',
       '597_', '617', '119', '663', '640', '92_', '1019', '501', '1302',
       '39', '716', '848', '931', '1214', '186', '424', '1001', '1110',
       '1152', '457', '1433', '1187', '52', '1480', '1047', '1035',
       '1347_', '33', '193', '699', '329', '1451', '484', '132', '649',
       '995', '545', '684', '1135', '1094', '1204', '654', '58', '348',
       '614', '1363', '323', '1406', '1348', '430', '153', '1461', '905',
       '1312', '1424', '1154', '95', '1353', '1228', '819', '1006', '795',
       '359', '1209', '590', '696', '1185_', '1465', '911', '1181', '70',
       '816', '1369', '143', '1416', '455', '55', '1096', '1474', '420',
       '1131', '904', '89', '1259', '527', '1241', '449', 

In [14]:
df['Delay_from_due_date'].unique() # no problem here

array([ 3, -1,  5,  6,  8,  7, 13, 10,  0,  4,  9,  1, 12, 11, 30, 31, 34,
       27, 14,  2, -2, 16, 17, 15, 23, 22, 21, 18, 19, 52, 51, 48, 53, 26,
       43, 28, 25, 20, 47, 46, 49, 24, 61, 29, 50, 58, 45, 59, 55, 56, 57,
       54, 62, 65, 64, 67, 36, 41, 33, 32, 39, 44, 42, 60, 35, 38, -3, 63,
       40, 37, -5, -4, 66], dtype=int64)

In [15]:
df['Num_of_Delayed_Payment'].unique()

array(['7', nan, '4', '8_', '6', '1', '-1', '3_', '0', '8', '5', '3', '9',
       '12', '15', '17', '10', '2', '2_', '11', '14', '20', '22', '13',
       '13_', '14_', '16', '12_', '18', '19', '23', '24', '21', '3318',
       '3083', '22_', '1338', '4_', '26', '11_', '3104', '21_', '25',
       '10_', '183_', '9_', '1106', '834', '19_', '24_', '17_', '23_',
       '2672', '20_', '2008', '-3', '538', '6_', '1_', '16_', '27', '-2',
       '3478', '2420', '15_', '707', '708', '26_', '18_', '3815', '28',
       '5_', '1867', '2250', '1463', '25_', '7_', '4126', '2882', '1941',
       '2655', '2628', '132', '3069', '306', '0_', '3539', '3684', '1823',
       '4128', '1946', '827', '2297', '2566', '904', '182', '929', '3568',
       '2503', '1552', '2812', '1697', '3764', '851', '3905', '923', '88',
       '1668', '3253', '808', '2689', '3858', '642', '3457', '1402',
       '1732', '3154', '847', '3037', '2204', '3103', '1063', '2056',
       '1282', '1841', '2569_', '211', '793', '3484', '4

In [16]:
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].str.replace('_','')
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].astype(float)

In [17]:
delayed_payment_by_customer =df.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform('mean')
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].fillna(delayed_payment_by_customer)

In [18]:
df['Num_Credit_Inquiries'].unique()

array([   4.,    2.,    3., ..., 1361.,  310.,   74.])

In [19]:
inquiries_by_customer =df.groupby('Customer_ID')['Num_Credit_Inquiries'].transform('mean')
df['Num_Credit_Inquiries'] = df['Num_Credit_Inquiries'].fillna(inquiries_by_customer)

In [20]:
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].replace('__10000__',np.NaN)
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].astype(float)

In [21]:
invested_by_customer =df.groupby('Customer_ID')['Amount_invested_monthly'].transform('mean')
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].fillna(invested_by_customer)

In [22]:
df['Monthly_Balance'] = df['Monthly_Balance'].replace('__-333333333333333333333333333__',np.NaN)
df['Monthly_Balance'] = df['Monthly_Balance'].astype(float)

In [23]:
df['Monthly_Balance'] = df['Monthly_Balance'].fillna(df.groupby('Customer_ID')['Monthly_Balance'].transform('mean'))

In [24]:
df['Changed_Credit_Limit'].unique()

array(['11.27', '_', '6.27', ..., '17.509999999999998', '25.16', '21.17'],
      dtype=object)

In [25]:
df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].replace('_',np.NaN)
df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].astype(float)

In [26]:
df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].fillna(df.groupby('Customer_ID')['Changed_Credit_Limit'].transform('mean'))

In [27]:
df['Outstanding_Debt'].unique()

array(['809.98', '605.03', '1303.01', ..., '3571.7_', '3571.7', '502.38'],
      dtype=object)

In [28]:
df['Outstanding_Debt'] = df['Outstanding_Debt'].str.replace('_','')
df['Outstanding_Debt'] = df['Outstanding_Debt'].astype(float)

In [29]:
df['Credit_Mix'].unique()


array(['_', 'Good', 'Standard', 'Bad'], dtype=object)

In [30]:
df['Credit_Mix'] = df['Credit_Mix'].replace('_',np.NaN)
df['Credit_Mix'] = df.groupby('Customer_ID')['Credit_Mix'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None) 


In [31]:
df['Occupation'].unique()


array(['Scientist', '_______', 'Teacher', 'Engineer', 'Entrepreneur',
       'Developer', 'Lawyer', 'Media_Manager', 'Doctor', 'Journalist',
       'Manager', 'Accountant', 'Musician', 'Mechanic', 'Writer',
       'Architect'], dtype=object)

In [32]:
df['Occupation'] = df['Occupation'].replace('_______',np.NaN)
df['Occupation'] = df.groupby('Customer_ID')['Occupation'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None) 

In [33]:
df['Payment_of_Min_Amount'].unique()

array(['No', 'NM', 'Yes'], dtype=object)

In [34]:
df['Credit_History_Age'] = df.groupby('Customer_ID')['Credit_History_Age'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None) 

In [35]:
df['Payment_Behaviour'].unique()

array(['High_spent_Small_value_payments',
       'Low_spent_Large_value_payments',
       'Low_spent_Medium_value_payments',
       'Low_spent_Small_value_payments',
       'High_spent_Medium_value_payments', '!@9#%8',
       'High_spent_Large_value_payments'], dtype=object)

In [36]:
df['Payment_Behaviour'] = df['Payment_Behaviour'].replace('!@9#%8',np.NaN)
df['Payment_Behaviour'] = df.groupby('Customer_ID')['Payment_Behaviour'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None) 

In [37]:
df = df.drop(['Credit_History_Age','ID','Customer_ID','Name','Age','Num_of_Loan','SSN'],axis=1)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  object 
 1   Occupation                100000 non-null  object 
 2   Annual_Income             100000 non-null  float64
 3   Monthly_Inhand_Salary     100000 non-null  float64
 4   Num_Bank_Accounts         100000 non-null  int64  
 5   Num_Credit_Card           100000 non-null  int64  
 6   Interest_Rate             100000 non-null  int64  
 7   Type_of_Loan              100000 non-null  object 
 8   Delay_from_due_date       100000 non-null  int64  
 9   Num_of_Delayed_Payment    100000 non-null  float64
 10  Changed_Credit_Limit      100000 non-null  float64
 11  Num_Credit_Inquiries      100000 non-null  float64
 12  Credit_Mix                100000 non-null  object 
 13  Outstanding_Debt          100000 non-null  fl

In [39]:
# we finished data cleaning now to change the string columns into incoded data


In [40]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [41]:
dummy = pd.get_dummies(x[['Month','Occupation','Type_of_Loan','Credit_Mix','Payment_of_Min_Amount','Payment_Behaviour']])

In [42]:
x = x.drop(['Month','Occupation','Type_of_Loan','Credit_Mix','Payment_of_Min_Amount','Payment_Behaviour'],axis=1)

In [43]:
x = pd.concat([x,dummy],axis=1)

In [44]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 51 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   Annual_Income                                       100000 non-null  float64
 1   Monthly_Inhand_Salary                               100000 non-null  float64
 2   Num_Bank_Accounts                                   100000 non-null  int64  
 3   Num_Credit_Card                                     100000 non-null  int64  
 4   Interest_Rate                                       100000 non-null  int64  
 5   Delay_from_due_date                                 100000 non-null  int64  
 6   Num_of_Delayed_Payment                              100000 non-null  float64
 7   Changed_Credit_Limit                                100000 non-null  float64
 8   Num_Credit_Inquiries                                100000 non-nu

In [48]:
# now for the prediction
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 

In [47]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)